<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/84_Pro_%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%BC%D1%83%D0%B7%D1%8B%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание Pro

Макс 10 баллов

Сравните результаты созданной в ходе занятия нейронной сети и:

Сети с одним LSTM слоем

Сети с двумя LSTM слоями

Сети с bi-directional LSTM вместо обычных LSTM

Проанализируйте результаты моделей и напишите результаты анализа в текстовой ячейке в конце Вашего блокнота. Обучайте каждую модель не менее пяти эпох.

Статья про Bidirectional https://proproprogs.ru/neural_network/bidirectional-rekurrentnye-neyronnye-seti

*Разбор данного раздела:* https://youtu.be/VS6DHal1xYw?t=467

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Flatten
from keras.callbacks import ModelCheckpoint
from google.colab import files, drive
from matplotlib import pyplot as plt
from music21 import *
import glob
import pickle
from IPython.display import clear_output
from IPython.display import Audio

In [ ]:
drive.mount('/content/drive') # Монтируем гугл диск

Mounted at /content/drive


# Как устроен midi файл

*Разбор данного раздела:* https://youtu.be/VS6DHal1xYw?t=581

In [ ]:
!unzip '/content/drive/MyDrive/Базы/Генерация_музыки/class.zip' -d './data/'
clear_output()

In [ ]:
midi = converter.parse('/content/data/alb_esp1.mid') # Функция парсит MIDI файл в удобный для работы формат
midi

<music21.stream.Score 0x7f2bcec78be0>

In [ ]:
notes_to_parse = midi.flat.notes  # Функция, позволяющая получить список нот и аккордов в файле
for element in notes_to_parse[0:40]:
  print(element, element.offset) # Оффсет(смещение) позволяет проигрывать ноты в разное время

<music21.note.Note A> 0.5
<music21.note.Note A> 0.5
<music21.note.Note E> 1.0
<music21.note.Note E> 1.0
<music21.note.Note D> 4.25
<music21.note.Note D> 4.25
<music21.note.Note E> 13/3
<music21.note.Note E> 13/3
<music21.note.Note D> 4.5
<music21.note.Note D> 4.5
<music21.note.Note C> 5.0
<music21.note.Note C> 5.0
<music21.note.Note D> 5.5
<music21.note.Note D> 5.5
<music21.note.Note E> 6.0
<music21.note.Note E> 6.0
<music21.note.Note F> 6.5
<music21.note.Note F> 6.5
<music21.note.Note G> 7.0
<music21.note.Note G> 7.0
<music21.note.Note E> 7.5
<music21.note.Note E> 7.5
<music21.note.Note F> 8.0
<music21.note.Note F> 8.0
<music21.note.Note E> 8.5
<music21.note.Note E> 8.5
<music21.note.Note D> 8.75
<music21.note.Note E> 8.75
<music21.note.Note D> 8.75
<music21.note.Note E> 8.75
<music21.note.Note D> 9.0
<music21.note.Note D> 9.0
<music21.note.Note C> 10.0
<music21.note.Note C> 10.0
<music21.note.Note B-> 11.0
<music21.note.Note B-> 11.0
<music21.note.Note A> 12.0
<music21.note.Note A> 1

In [ ]:
s2 = instrument.partitionByInstrument(midi) # Если в файле есть несколько инструментов
s2.parts[0] # Берем только часть с пианино

<music21.stream.Part Piano>

## Собираем список нот

*Разбор данного раздела:* https://youtu.be/VS6DHal1xYw?t=858

In [ ]:
notes = [] # Здесь будут храниться все ноты из всех файлов
for i, file in enumerate(glob.glob("/content/data/*.mid")):
  midi = converter.parse(file) 
  print('\r', 'Обрабатывается файл', i, " ",file, end='')
  notes_to_parse = None
  try: # Если в файле нет разных иструментов 
    notes_to_parse = midi.flat.notes
  except: # Если есть разные инструменты
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() # Итератор


  for element in notes_to_parse: # Добавляем в список ноту или аккорд
    if isinstance(element, note.Note): # Если нота, просто добавляем ее
      notes.append(str(element.pitch)) 
    elif isinstance(element, chord.Chord): # Если аккорд, то записываем числовой вариант всех нот через точку
      notes.append('.'.join(str(n) for n in element.normalOrder))

 Обрабатывается файл 80   /content/data/chpn-p6.mid

In [ ]:
notes_ = []
midi = converter.parse('/content/data/alb_esp1.mid') 
s2 = instrument.partitionByInstrument(midi)
notes_to_parse = s2.parts[0].recurse() # Итератор
for element in notes_to_parse: # Добавляем в список ноту или аккорд
    # if isinstance(element, note.Note): # Если нота, просто добавляем ее
    #     notes_.append(str(element.pitch)) 
    if isinstance(element, chord.Chord): # Если аккорд, то записываем числовой вариант всех нот через точку
        notes_.append('.'.join(str(n) for n in element.normalOrder))

In [ ]:
notes_[:10]

['1.4', '10.2.4', '1.4', '10.2.4', '4.10', '2.8', '4.9', '9.1.4']

## Сохраняем полученный список на диск и загружаем

*Разбор данного раздела:* https://youtu.be/VS6DHal1xYw?t=1200




Сохранение

In [ ]:
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

Загрузка

In [ ]:
with open ('/content/notes', 'rb') as file:
  notes = pickle.load(file)

# Подготовка данных

In [ ]:
notes[510:520]

['6.9', '6.9', '6.9', '3.9', '6.9.0', '6.9.0', '4', '7.11', '7.11', '7.11']

In [ ]:
len(notes)

139269

In [ ]:
n_list = set(notes) # Уникальные ноты и аккорды
len(n_list) # Их количество

351

In [ ]:
sequence_length = 100 # длина последовательности нот/аккордов, по которой сеть будет делать предсказания

# Получаем отсортированный список всех уникальных нот и аккордов
pitchnames = sorted(n_list)

# Создаем словарь, сопоставляющий ноту/аккорд числу(int)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 

network_input = [] # Список со всеми последовательностями
network_output = [] # Список со всеми элементами, идущими после последовательности

# Создаем обучающую выборку
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length] # Выбираем последовательность длины sequence_length
  sequence_out = notes[i + sequence_length]  # Следующий за последовательностью элемент
  network_input.append([note_to_int[char] for char in sequence_in]) # Добавляем в список последовательность, предварительно переведя все ноты/аккорды в числа
  network_output.append(note_to_int[sequence_out]) # Добавляем в список следующий за последовательностью элемент, переведя его в число

n_patterns = len(network_input) # Количество полученных последовательностей

# Делаем решейп, чтобы сеть могла обучаться
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# Нормализируем входные данные
network_input = network_input / float(len(n_list))
# Выходые данные преобразуем в one-hot encoding
network_output = np.array(network_output)
# network_output = keras.utils.to_categorical(network_output)

In [ ]:
network_input.shape

(139169, 100, 1)

# Архитектура сети

*Разбор данного раздела:* https://youtu.be/VS6DHal1xYw?t=1557

In [ ]:
def create_network(network_input, n_list, dropout=0.3):

    model = Sequential()
    model.add(LSTM(
                        256,
                        input_shape=(network_input.shape[1], network_input.shape[2]),
                        return_sequences=True 
                        ))
    model.add(Dropout(dropout))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(256))
    model.add(Dense(128))
    model.add(Dropout(dropout))
    model.add(Dense(len(n_list)))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [ ]:
model = create_network(network_input, n_list)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          264192    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 256)          525312    
                                                                 
 dropout_1 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0

Для обучения сети использовался ModelCheckpoint, позволяющий сохранять веса в процессе обучения

In [ ]:
filepath = "/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"    # Где и с каким именем сохраняются веса
checkpoint = ModelCheckpoint(filepath,
                            monitor='loss',
                            verbose=0,
                            save_weights_only = True, # Сохраняем только веса   
                            save_best_only=True,      # Сохраняем только при уменьшении ошибки
                            mode='min',
                            )    
callbacks_list = [checkpoint]   

model.fit(network_input, network_output, epochs=20, 
          batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1088/1088 [==============================] - 60s 55ms/step - loss: 4.5451 - accuracy: 0.0298
Epoch 2/20
1088/1088 [==============================] - 55s 51ms/step - loss: 4.5228 - accuracy: 0.0317
Epoch 3/20
1088/1088 [==============================] - 58s 53ms/step - loss: 4.4894 - accuracy: 0.0365
Epoch 4/20
1088/1088 [==============================] - 57s 52ms/step - loss: 4.4539 - accuracy: 0.0407
Epoch 5/20
1088/1088 [==============================] - 57s 52ms/step - loss: 4.4175 - accuracy: 0.0450
Epoch 6/20
1088/1088 [==============================] - 59s 54ms/step - loss: 4.3734 - accuracy: 0.0502
Epoch 7/20
1088/1088 [==============================] - 55s 51ms/step - loss: 4.3211 - accuracy: 0.0581
Epoch 8/20
1088/1088 [==============================] - 55s 50ms/step - loss: 4.2510 - accuracy: 0.0679
Epoch 9/20
1088/1088 [==============================] - 56s 51ms/step - loss: 4.1752 - accuracy: 0.0782
Epoch 10/20
1088/1088 [==============================] - 57s 52m

# Иссл.№1. Сеть с одним LSTM слоем


In [ ]:
def create_network_LSTM_1(network_input, n_list, dropout=0.3):

    model = Sequential()
    model.add(LSTM(
                        256,
                        input_shape=(network_input.shape[1], network_input.shape[2]),
                        return_sequences=True 
                        ))
    model.add(Dropout(dropout))    
    model.add(Dense(128))
    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(len(n_list)))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [ ]:
model_LSTM_1 = create_network_LSTM_1(network_input, n_list)
model_LSTM_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 100, 256)          264192    
                                                                 
 dropout_8 (Dropout)         (None, 100, 256)          0         
                                                                 
 dense_6 (Dense)             (None, 100, 128)          32896     
                                                                 
 dropout_9 (Dropout)         (None, 100, 128)          0         
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense_7 (Dense)             (None, 351)               4493151   
                                                                 
 activation_3 (Activation)   (None, 351)              

Для обучения сети использовался ModelCheckpoint, позволяющий сохранять веса в процессе обучения

In [ ]:
filepath = "/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement-LSTM_1-{epoch:02d}-{loss:.4f}.hdf5"    # Где и с каким именем сохраняются веса
checkpoint = ModelCheckpoint(filepath,
                            monitor='loss',
                            verbose=0,
                            save_weights_only = True, # Сохраняем только веса   
                            save_best_only=True,      # Сохраняем только при уменьшении ошибки
                            mode='min',
                            )    
callbacks_list = [checkpoint]   

model_LSTM_1.fit(network_input, network_output, epochs=20, 
          batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1088/1088 [==============================] - 23s 19ms/step - loss: 4.6153 - accuracy: 0.0262
Epoch 2/20
1088/1088 [==============================] - 21s 20ms/step - loss: 4.5227 - accuracy: 0.0350
Epoch 3/20
1088/1088 [==============================] - 20s 18ms/step - loss: 4.4907 - accuracy: 0.0360
Epoch 4/20
1088/1088 [==============================] - 23s 21ms/step - loss: 4.4738 - accuracy: 0.0373
Epoch 5/20
1088/1088 [==============================] - 20s 18ms/step - loss: 4.4594 - accuracy: 0.0382
Epoch 6/20
1088/1088 [==============================] - 20s 19ms/step - loss: 4.4461 - accuracy: 0.0403
Epoch 7/20
1088/1088 [==============================] - 20s 18ms/step - loss: 4.4295 - accuracy: 0.0417
Epoch 8/20
1088/1088 [==============================] - 20s 18ms/step - loss: 4.4044 - accuracy: 0.0444
Epoch 9/20
1088/1088 [==============================] - 20s 18ms/step - loss: 4.3811 - accuracy: 0.0473
Epoch 10/20
1088/1088 [==============================] - 20s 18m

# Иссл.№2. Сеть с двумя LSTM слоями

In [ ]:
def create_network_LSTM_2(network_input, n_list, dropout=0.3):

    model = Sequential()
    model.add(LSTM(
                        256,
                        input_shape=(network_input.shape[1], network_input.shape[2]),
                        return_sequences=True 
                        ))
    model.add(Dropout(dropout))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(Flatten())    
    model.add(Dense(128))
    model.add(Dropout(dropout))
    model.add(Dense(len(n_list)))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [ ]:
model_LSTM_2 = create_network_LSTM_2(network_input, n_list)
model_LSTM_2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100, 256)          264192    
                                                                 
 dropout_10 (Dropout)        (None, 100, 256)          0         
                                                                 
 lstm_9 (LSTM)               (None, 100, 256)          525312    
                                                                 
 dropout_11 (Dropout)        (None, 100, 256)          0         
                                                                 
 flatten_1 (Flatten)         (None, 25600)             0         
                                                                 
 dense_8 (Dense)             (None, 128)               3276928   
                                                                 
 dropout_12 (Dropout)        (None, 128)              

Для обучения сети использовался ModelCheckpoint, позволяющий сохранять веса в процессе обучения

In [ ]:
filepath = "/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement-LSTM_2-{epoch:02d}-{loss:.4f}.hdf5"    # Где и с каким именем сохраняются веса
checkpoint = ModelCheckpoint(filepath,
                            monitor='loss',
                            verbose=0,
                            save_weights_only = True, # Сохраняем только веса   
                            save_best_only=True,      # Сохраняем только при уменьшении ошибки
                            mode='min',
                            )    
callbacks_list = [checkpoint]   

model_LSTM_2.fit(network_input, network_output, epochs=20, 
          batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1088/1088 [==============================] - 41s 34ms/step - loss: 4.6418 - accuracy: 0.0251
Epoch 2/20
1088/1088 [==============================] - 36s 33ms/step - loss: 4.5592 - accuracy: 0.0300
Epoch 3/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.5410 - accuracy: 0.0328
Epoch 4/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.5282 - accuracy: 0.0351
Epoch 5/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.5150 - accuracy: 0.0363
Epoch 6/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.5075 - accuracy: 0.0372
Epoch 7/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.4965 - accuracy: 0.0383
Epoch 8/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.4772 - accuracy: 0.0415
Epoch 9/20
1088/1088 [==============================] - 37s 34ms/step - loss: 4.4651 - accuracy: 0.0415
Epoch 10/20
1088/1088 [==============================] - 37s 34m

# Иссл.№3. Сеть с bi-directional LSTM вместо обычных LSTM

In [ ]:
from keras.layers import Bidirectional

In [ ]:
def create_network_Bidirectional_LSTM(network_input, n_list, dropout=0.3):

    model = Sequential()
    model.add(Bidirectional(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True)))
    model.add(Dropout(dropout))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(dropout))
    model.add(Bidirectional(LSTM(256)))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Dropout(dropout))
    model.add(Dense(len(n_list)))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [ ]:
model_Bidirectional_LSTM = create_network_Bidirectional_LSTM(network_input, n_list)
# model_Bidirectional_LSTM.summary()

Для обучения сети использовался ModelCheckpoint, позволяющий сохранять веса в процессе обучения

In [ ]:
filepath = "/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement-Bidirectional_LSTM-{epoch:02d}-{loss:.4f}.hdf5"    # Где и с каким именем сохраняются веса
checkpoint = ModelCheckpoint(filepath,
                            monitor='loss',
                            verbose=0,
                            save_weights_only = True, # Сохраняем только веса   
                            save_best_only=True,      # Сохраняем только при уменьшении ошибки
                            mode='min',
                            )    
callbacks_list = [checkpoint]   

model_Bidirectional_LSTM.fit(network_input, network_output, epochs=20, 
          batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1088/1088 [==============================] - 237s 124ms/step - loss: 4.6323 - accuracy: 0.0242
Epoch 2/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.5648 - accuracy: 0.0277
Epoch 3/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.5400 - accuracy: 0.0302
Epoch 4/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.5193 - accuracy: 0.0330
Epoch 5/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.4978 - accuracy: 0.0356
Epoch 6/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.4711 - accuracy: 0.0385
Epoch 7/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.4388 - accuracy: 0.0431
Epoch 8/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.3965 - accuracy: 0.0486
Epoch 9/20
1088/1088 [==============================] - 136s 125ms/step - loss: 4.3379 - accuracy: 0.0575
Epoch 10/20
1088/1088 [=======================

# Выводы

Лучший результат точности и ошибки показала созданная в ходе занятия нейронная сеть: loss = 3.4240 , accuracy = 0.1807, время на эпоху = 52с. 

В 2 раза быстрее обучилась сеть с одним LSTM слоем и результат точности оказался немного хуже, а ошибка меньше:  loss = 3.9694 , accuracy = 0.1136, время на эпоху = 20с.

Немного получше результат точности и ошибки у сети с Bidirectional_LSTM слоями , но врмя, затраченное на одну эпоху = 136с, что в разы больше данного показателя у других сетей, loss = 3.4134 , accuracy = 0.1833.

Хуже всех оказался результат у сети с 2-мя LSTM слоями. Возможно на большом количестве эпох она дала бы лучший результат. 

Обобщая выше изложенные результаты можно сделать вывод, что использование Bidirectional_LSTM слоев требует очень больших ресурсов, а лучший результат может дать однослойная LSTM сеть. И может быть следует брать окно поменьше для обработки признаков, так будет побыстрее, но это не точноПоэтому нужно начинать исследования от простого к сложному. 

